In [1]:
import numpy as np


In [2]:
a = np.array(12)

用$r$表示浮点实数，$q$表示量化后的定点整数。浮点和整型之间的换算公式为:

$$
r = S(q-Z) \tag{1}
$$


$$
q = round(\frac{r}{S}+Z) \tag{2}
$$

$s$ 是 scale，表示实数和整数之间的比例关系.

$$
S = \frac{r_{max}-r_{min}}{q_{max}-q_{min}}\tag{3}
$$


$z$是 zero point，表示实数中的 0 经过量化后对应的整数。

$$
Z = round(q_{max} - \frac{r_{max}}{S})\tag{4}
$$



In [9]:
#比如浮点数 的范围是[-0.2,0.2]
#要做int8的量化，那么Qmax=127     Qmin=-127
Qmax = 127
Qmin = -127
Rmin = -0.2
Rmax = 0.2

scale = (Rmax - Rmin) / (Qmax - Qmin)
zero_point = np.round(Qmax - Rmax/S)

print("scale:", scale)
print("zeropoint:",zero_point)
r1 = 0.135
q1 = np.round(r1/scale + zero_point)
print(r1,"的量化值为:",q1)


scale: 0.0015748031496062994
zeropoint: 0.0
0.135 的量化值为: 86.0


假设 $S_1$、$Z_1$ 是 $r_1$、$r_2$ 数对应的 scale 和 zero point， $S_2$、$Z_2$、$S_3$、$Z_3$同理

$$
S_{3}(q_{3}-Z_{3})=S_1(q_{1}-Z_{1})S_{2}(q_{2}-Z_{2})  \tag{6}
$$

那么
$$
q_{3}=\frac{S_{1} S_{2}}{S_{3}}(q_{1}-Z_{1})(q_{2}-Z_{2}) + Z_{3}  \tag{7}
$$



In [57]:
def quant(r,s,z):
    q = np.round( r/s  +  z)
    return q

Qmax = 127
Qmin = -127


R1min = -0.1
R1max = 0.2
r1 = 0.19
s1 =  (R1max - R1min) / (Qmax - Qmin)
z1 =  np.round(Qmax - R1max/s1)
q1 =quant(r1,s1,z1)



R2min = -0.15
R2max = 0.12
r2 = -0.112
s2 =  (R2max - R2min) / (Qmax - Qmin)
z2 =  np.round(Qmax - R2max/s2)
q2 =quant(r2,s2,z2)



R3min = -0.05
R3max = 0.05
r3 = r1 * r2
s3 =  (R3max - R3min) / (Qmax - Qmin)
z3 =  np.round(Qmax - R3max/s3)
q3 =quant(r3,s3,z3)

print(q1)
print(q2)
print(q3)


q3_1 = (s1 * s2 / s3) * ( q1 - z1 ) * ( q2 - z2 ) + z3

print(q3_1)
print(np.round(q3_1))


#量化的是否可以不一样呢？  比如q1为uint8量化   q2为int8量化  



119.0
-91.0
-54.0
-53.90964566929136
-54.0


In [56]:
Q1max = 127
Q1min = -127


R1min = -0.1
R1max = 0.2
r1 = 0.19
s1 =  (R1max - R1min) / (Q1max - Q1min)
z1 =  np.round(Q1max - R1max/s1)
q1 =quant(r1,s1,z1)


Q2max = 0
Q2min = 255

R2min = -0.15
R2max = 0.12
r2 = 0.112
s2 =  (R2max - R2min) / (Q2max - Q2min)
z2 =  np.round(Q2max - R2max/s2)
q2 =quant(r2,s2,z2)


Q3max = 127
Q3min = -127

R3min = -0.05
R3max = 0.05
r3 = r1 * r2
s3 =  (R3max - R3min) / (Q3max - Q3min)
z3 =  np.round(Q3max - R3max/s3)
q3 =quant(r3,s3,z3)

print(q1)
print(q2)
print(q3)


q3_1 = (s1 * s2 / s3) * ( q1 - z1 ) * ( q2 - z2 ) + z3
print(q3_1)
print(np.round(q3_1))

119.0
7.0
54.0
54.20964705882354
54.0


观察一下上面的公式，会发现除了$\frac{S_1 S_2}{S_3}$，浮点数运算，其他都是定点整数运算。要知道，在一些专用计算设备上是不支持浮点的计算的，那是否如何把$\frac{S_1 S_2}{S_3}$也变成定点运算呢？

假设 $M=\frac{S_1 S_2}{S_3}$，由于 $M$ 通常都是 (0, 1) 之间的实数 (这是通过大量实验统计出来的)，因此可以表示成 $M=2^{-n}M_0$，其中 $M_0$ 是一个定点实数。注意，定点数并不一定是整数，所谓定点，指的是小数点的位置是固定的，即小数位数是固定的。因此，如果存在 $M=2^{-n}M_0$，那我们就可以通过 $M_0$ 的 bit 位移操作实现 $2^{-n}M_0$，这样整个过程就都在定点上计算了。




假设 $P=7091$，$M=0.0072474273418460$ ($M$可以通过 $S$ 事先计算得到)，那下面我们就是要找到一个 $M0$ 和$n$，使得 $MP=2^{-n}M_0 P$ 成立。我们可以用一段代码来找到这两个数：

In [6]:
M = 0.0072474273418460
P = 7091

def multiply(n, M, P):
    result = M * P
    Mo = int(round(2 ** n * M)) # 这里不一定要四舍五入截断，因为python定点数不好表示才这样处理

    approx_result = (Mo * P) >> n
    print("n=%d, Mo=%d, approx=%f, error=%f"%\
          (n, Mo, approx_result, result-approx_result))

for n in range(1, 16):
    multiply(n, M, P)

n=1, Mo=0, approx=0.000000, error=51.391507
n=2, Mo=0, approx=0.000000, error=51.391507
n=3, Mo=0, approx=0.000000, error=51.391507
n=4, Mo=0, approx=0.000000, error=51.391507
n=5, Mo=0, approx=0.000000, error=51.391507
n=6, Mo=0, approx=0.000000, error=51.391507
n=7, Mo=1, approx=55.000000, error=-3.608493
n=8, Mo=2, approx=55.000000, error=-3.608493
n=9, Mo=4, approx=55.000000, error=-3.608493
n=10, Mo=7, approx=48.000000, error=3.391507
n=11, Mo=15, approx=51.000000, error=0.391507
n=12, Mo=30, approx=51.000000, error=0.391507
n=13, Mo=59, approx=51.000000, error=0.391507
n=14, Mo=119, approx=51.000000, error=0.391507
n=15, Mo=237, approx=51.000000, error=0.391507


可以看到，在 $n=11$、$M0=15$ 的时候，误差就已经在 1 以内了。因此，只要 $M_0P$的数值范围在 21(32-11) 个 bit 内，就可以通过对 $M_0P$右移 $n$ 个 bit 来近似 $MP$ 了，而这个误差本身在可以接受的范围内。这样一来，(8) 式就可以完全通过定点运算来计算，即我们实现了浮点矩阵乘法的量化。

In [ ]:
multiper   和shift 怎么计算？？


add